In [1]:
include("optim_lib.jl")
include("lib_func.jl")
using Optim


# Constants and Parameters
d1, d2, ds = 3, 8, 1  # Degree range and step
const n, a, b = 4, 5, 1 
const C = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alph = .9 , 2 / 10  # Sampling parameters
f = camel_4d # Objective function

camel_4d (generic function with 1 method)

In [2]:
coeffs_poly_approx = main_gen(f, n, d1, d2, ds, delta, alph, C, 0.2)
# precis_poly_approx = precise_gen(f, n, d1, d2, ds, delta, alph, C, 0.2) ## test ##

dimension Vector space: 35
sample size: 256
dimension Vector space: 70
sample size: 256
dimension Vector space: 126
sample size: 625
dimension Vector space: 210
sample size: 1296
dimension Vector space: 330
sample size: 1296
dimension Vector space: 495
sample size: 2401


6-element Vector{Vector{Float64}}:
 [4145.833333333335, -2.317142615357207e-13, 1672.3958333333346, -2.1204834508992931e-13, 1.0606643163714152e-13, 25.000000000000245, 7.242581278422797e-14, 1199.9999999999998, 8.264355089816586e-14, -1.7978193187324755e-15  …  -7.230576856378686e-14, 25.000000000000274, 2.0949978219851147e-14, 1.3515396060871498e-14, -4.832711963622849e-14, 1200.0000000000002, 2.9161289446345647e-13, -2.3158687969552335e-13, -1.8195533877561802e-13, -2.2305920654212812e-13]
 [-3996.962678513309, -1760.9799883590883, -1581.4709034805144, 4251.381770964122, -2.7632060509706945e19, -751.316091389602, 25.000000000000398, 9.761781627881617e-14, 2.5070469797458175e-13, -188.25111856246485  …  3.7263126256871475e-15, 2.1395440155939308e-13, 1.5058209300425507e-13, -2.2095941705952675e-15, 3.7744742683546683e-13, -179.4696207938146, 1.792885954548191e-13, 5.2289051535886085e-14, 1.500118270776763e-12, 1.1664714411908134e18]
 [3350.230252756716, -1060.464517621262, 1654.70486

In [12]:
println(coeffs_poly_approx)
# println(precis_poly_approx[2]) 

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [4]:


using DynamicPolynomials, MultivariatePolynomials, HomotopyContinuation, ProgressLogging, DataFrames, Optim

include("hom_solve.jl") # Include the homotopy solver and main function
vars = @polyvar(x[1:n]) # Define polynomial ring 
h_x, h_y, h_z, h_t, col = main_4d(n, d1, d2, ds, coeffs_poly_approx) # main_4d is in hom_solve.jl
df = DataFrame(x=C * h_x, y=C * h_y, z=C * h_z, t=C * h_t, col=col)
df[!, :result] = [f([df.x[i], df.y[i], df.z[i], df.t[i]]) for i in 1:nrow(df)]
df[!, :distance_to_minima] = zeros(nrow(df))
df[!, :steps] = zeros(Int, nrow(df))
df[!, :converged] = falses(nrow(df))

println(df)

Computing mixed cells... 2    Time: 0:00:00
Computing mixed cells... 4    Time: 0:00:01
  mixed_volume:  16


Tracking 16 paths...  12%|███▊                          |  ETA: 0:01:11



Tracking 16 paths... 100%|██████████████████████████████| Time: 0:00:11
  # paths tracked:                  16
  # non-singular solutions (real):  16 (8)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         16 (8)
Degree: 3
Number of solutions: 1
Tracking 81 paths...   2%|▊                             |  ETA: 0:03:35



Tracking 81 paths... 100%|██████████████████████████████| Time: 0:00:05
  # paths tracked:                  81
  # non-singular solutions (real):  81 (81)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         81 (81)
Degree: 4
Number of solutions: 81
Tracking 256 paths...   1%|▎                            |  ETA: 0:15:13



Tracking 256 paths...  27%|███████▊                     |  ETA: 0:00:20



Tracking 256 paths...  52%|███████████████              |  ETA: 0:00:07



Tracking 256 paths...  75%|█████████████████████▋       |  ETA: 0:00:03



Tra

In [5]:


for i in 1:nrow(df)
    println("Optimizing for point $i")
    x0 = [df.x[i], df.y[i], df.z[i], df.t[i]]
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)

    distance = norm(x0 - minimizer)

    df.distance_to_minima[i] = distance
    df.steps[i] = steps
    df.converged[i] = converged

    println("Initial point: ", x0)
    println("Minimizer: ", minimizer)
    println("Distance to minima: ", distance)
    println("Number of steps: ", steps)
    println("Converged: ", converged)
    println(summary(res))
end

println(df)

Optimizing for point 1
Iter     Function value   Gradient norm 
     0    -4.386251e-30     6.533105e-15
 * time: 0.020483016967773438
Initial point: [7.379020605317168e-17, -6.508305728350678e-16, -2.1655633393074858e-17, -8.193447953142806e-16]
Minimizer: [7.379020605317168e-17, -6.508305728350678e-16, -2.1655633393074858e-17, -8.193447953142806e-16]
Distance to minima: 0.0
Number of steps: 0
Converged: true
L-BFGS
Optimizing for point 2
Iter     Function value   Gradient norm 
     0     1.920833e+03     7.654431e+02
 * time: 1.2874603271484375e-5
     1     5.686852e+02     3.417921e+02
 * time: 0.26340699195861816
     2     1.290281e+00     3.597028e+00
 * time: 0.263624906539917
     3    -1.295734e-01     1.429806e+00
 * time: 0.2636878490447998
     4    -3.570841e-01     1.032467e+00
 * time: 0.26374101638793945
     5    -4.213354e-01     4.185087e-01
 * time: 0.2637908458709717
     6    -4.308140e-01     3.720770e-02
 * time: 0.263840913772583
     7    -4.309276e-01     1

Excessive output truncated after 524291 bytes.

Initial point: [3.5355339059327378, 1.6368770376893263e-16, 3.5355339059327378, 

In [9]:
println(df)